# Interactive FastChem Demo

Define an atmosphere's temperature and pressure:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, widgets, VBox, HBox
from shone.chemistry.fastchem import FastchemWrapper

pressure = np.geomspace(1e-6, 1e2, 15)
temperature = 2300 * (np.geomspace(1e-6, 10, pressure.size) / 0.1) ** 0.1

ax = plt.gca()
ax.semilogy(temperature, pressure)
ax.invert_yaxis()
ax.set_xlabel('Temperature [K]')
ax.set_ylabel('Pressure [bar]')
ax.set_title('A T-P profile');

We create a FastChem wrapper like this, with default values:

In [ ]:
chem = FastchemWrapper(temperature, pressure, metallicity=0.7, c_to_o_ratio=1.2)

Now let's make an interactive explorer to plot abundance as a function of pressure for the species in FastChem. With the T-P profile set, this allows varying the metallicity and C/O ratio, plotting either the VMR or the MMR, given a free mean molecular weight.
<br><br>
<b>Note how for each combination of parameters, the chemistry is re-computed on the fly!</b>

In [ ]:
table = chem.get_species()

def plot_vmr(log_metallicity, c_to_o_ratio, species='O1Ti1', mmr=False, mean_molecular_weight=2.4):
    chem.metallicity = 10 ** log_metallicity
    chem.c_to_o_ratio = c_to_o_ratio
    
    if mmr: 
        mmr = chem.mmr_mmw() / mean_molecular_weight
        mixing_ratio = mmr[:, table.loc[species]['index']]
        xlabel = 'MMR'
    else:
        vmr = chem.vmr()
        mixing_ratio = vmr[:, table.loc[species]['index']]
        xlabel = 'VMR'

    plt.loglog(mixing_ratio, pressure)
    ax = plt.gca()
    ax.set(
        xlabel=xlabel,
        ylabel='Pressure [bar]'
    )
    ax.invert_yaxis()
    
dropdown_options = sorted(list(zip(
    list(table['name']), list(table['symbol'])
)), key=lambda x: x[0])[5:]

widget = interactive(
    plot_vmr,
    log_metallicity=(-1, 1, 0.1),
    c_to_o_ratio=(0.1, 1.2, 0.01),
    species=dropdown_options,
    mean_molecular_weight=(1, 10, 1),#Note that this is inactive if VMR is plotted.

)

controls = VBox(widget.children[:-1])
output = widget.children[-1]
display(HBox([output, controls]))